Here is a notebook that shows you how to build some simple mth5 files from one staiton.

Multistation archives are very similar and will be addressed in another ipynb.

In this case we focus on station "CAS04".  This example is being ported from 
aurora/tests/cas04/01_make_cas04_mth5.py

In [18]:
#Imports
import pandas as pd
from aurora.sandbox.mth5_channel_summary_helpers import channel_summary_to_make_mth5
from mth5.clients.make_mth5 import MakeMTH5
from mth5.utils.helpers import initialize_mth5
from mth5.utils.helpers import read_back_data
from mt_metadata.timeseries.stationxml import XMLInventoryMTExperiment

In [2]:
import logging, sys
logging.disable(sys.maxsize)


In [3]:
#placeholder for controlling which acquistions runs to request
#Leave it empty to get all of them
active_runs = []
#active_runs = ["a",]
#active_runs = ["b", "c", "d"]


In [4]:
maker = MakeMTH5(mth5_version="0.1.0")
maker.client = "IRIS"

In [10]:
# Generate data frame of FDSN
# Network, Station, Location, Channel, Startime, Endtime codes of interest
network = "8P"
station = "CAS04"
channels = ["LQE", "LQN", "LFE", "LFN", "LFZ", ]
start = "2020-06-02T19:00:00"
end = "2020-07-13T19:00:00"

request_list = []
for channel in channels:
    request = [network, station, "", channel, start, end]
    request_list.append(request)

print(f"Request List \n {request_list}")

# Turn list into dataframe
metadata_request_df = pd.DataFrame(request_list, columns=maker.column_names)
print(f"\n\n metadata_request_df \n {metadata_request_df}")


Request List 
 [['8P', 'CAS04', '', 'LQE', '2020-06-02T19:00:00', '2020-07-13T19:00:00'], ['8P', 'CAS04', '', 'LQN', '2020-06-02T19:00:00', '2020-07-13T19:00:00'], ['8P', 'CAS04', '', 'LFE', '2020-06-02T19:00:00', '2020-07-13T19:00:00'], ['8P', 'CAS04', '', 'LFN', '2020-06-02T19:00:00', '2020-07-13T19:00:00'], ['8P', 'CAS04', '', 'LFZ', '2020-06-02T19:00:00', '2020-07-13T19:00:00']]


 metadata_request_df 
   network station location channel                start                  end
0      8P   CAS04              LQE  2020-06-02T19:00:00  2020-07-13T19:00:00
1      8P   CAS04              LQN  2020-06-02T19:00:00  2020-07-13T19:00:00
2      8P   CAS04              LFE  2020-06-02T19:00:00  2020-07-13T19:00:00
3      8P   CAS04              LFN  2020-06-02T19:00:00  2020-07-13T19:00:00
4      8P   CAS04              LFZ  2020-06-02T19:00:00  2020-07-13T19:00:00


In [11]:
# Request the inventory information from IRIS
inventory, traces = maker.get_inventory_from_df(metadata_request_df, data=False)
translator = XMLInventoryMTExperiment()
experiment = translator.xml_to_mt(inventory_object=inventory)


In [12]:
h5_path = "tmp.h5"
mth5_obj = initialize_mth5(h5_path)  # mode="a")
mth5_obj.from_experiment(experiment)
mth5_obj.channel_summary.summarize()

summary_df = mth5_obj.channel_summary.to_dataframe()

In [13]:
print(summary_df)

         survey station run   latitude   longitude  elevation component  \
0   CONUS South   CAS04   a  37.633351 -121.468382   329.3875        ex   
1   CONUS South   CAS04   a  37.633351 -121.468382   329.3875        ey   
2   CONUS South   CAS04   a  37.633351 -121.468382   329.3875        hx   
3   CONUS South   CAS04   a  37.633351 -121.468382   329.3875        hy   
4   CONUS South   CAS04   a  37.633351 -121.468382   329.3875        hz   
5   CONUS South   CAS04   b  37.633351 -121.468382   329.3875        ex   
6   CONUS South   CAS04   b  37.633351 -121.468382   329.3875        ey   
7   CONUS South   CAS04   b  37.633351 -121.468382   329.3875        hx   
8   CONUS South   CAS04   b  37.633351 -121.468382   329.3875        hy   
9   CONUS South   CAS04   b  37.633351 -121.468382   329.3875        hz   
10  CONUS South   CAS04   c  37.633351 -121.468382   329.3875        ex   
11  CONUS South   CAS04   c  37.633351 -121.468382   329.3875        ey   
12  CONUS South   CAS04  

In [14]:
# <TRANSFORM CHANNEL SUMMARY INTO REQUEST DF>
if active_runs:
    summary_df = summary_df[summary_df["run"].isin(active_runs)]  # summary_df[0:5]
request_df = channel_summary_to_make_mth5(summary_df, network=network, verbose=True)
print(f"\n\n {request_df}")

('CAS04', 'a'), from 2020-06-02 18:41:43+00:00, to 2020-06-02 22:07:46+00:00
('CAS04', 'b'), from 2020-06-02 22:24:55+00:00, to 2020-06-12 17:52:23+00:00
('CAS04', 'c'), from 2020-06-12 18:32:17+00:00, to 2020-07-01 17:32:59+00:00
('CAS04', 'd'), from 2020-07-01 19:36:55+00:00, to 2020-07-13 21:46:12+00:00


    network station location channel                     start  \
0       8P   CAS04              LQN 2020-06-02 18:41:43+00:00   
1       8P   CAS04              LQE 2020-06-02 18:41:43+00:00   
2       8P   CAS04              LFN 2020-06-02 18:41:43+00:00   
3       8P   CAS04              LFE 2020-06-02 18:41:43+00:00   
4       8P   CAS04              LFZ 2020-06-02 18:41:43+00:00   
5       8P   CAS04              LQN 2020-06-02 22:24:55+00:00   
6       8P   CAS04              LQE 2020-06-02 22:24:55+00:00   
7       8P   CAS04              LFN 2020-06-02 22:24:55+00:00   
8       8P   CAS04              LFE 2020-06-02 22:24:55+00:00   
9       8P   CAS04              LFZ 202

In [16]:
mth5_version = "0.1.0"
interact = False
#mth5_version = "0.2.0"
maker = MakeMTH5(mth5_version=mth5_version)
# print("FAILED FOR 0.2.0 with some other error")
# inventory, streams = maker.get_inventory_from_df(request_df, data=False, client="IRIS")    # inventory==inventory0??
mth5_obj = maker.make_mth5_from_fdsnclient(request_df, client="IRIS", path="", interact=interact)
mth5_path = mth5_obj.filename

More or less runs have been requested by the user than are defined in the metadata. Runs will be defined but only the requested run extents contain time series data based on the users request.


In [17]:
if interact:
    mth5_obj.close_mth5()

In [ ]:
for run in active_runs:
    read_back_data(mth5_path, "CAS04", run)